## Compiling result for the fail fast experiments of the metakrr_mk using easy harmonic datasets

### Abstract
In this experiement, we compare MetaKRR single kernel (SK) with MetaKRR multi kernel (MK) using the meta-dataset easy harmonics.
We also explore different versions of MK to determine what is the best way to condition on a task.

Let's $z$ be the task representation. 
Two alternatives were explored to get $z$ herein :
- predefined task descriptor given by the data distribution that generates the task
- encoding of the training set of each episode/task using a neural network

And three methods were considered to adapt the shared feature extractor $\phi(x)$ to each task/episode:
+ fusion net $f([\phi(x); z])$: where $[a; b]$ is the concatenation of the vectors $a$ and $b$; and $f$ is neural network 
+ gated modulation: $f(\phi(x), z) = \phi(x) \cdot z $ where $\cdot$ is the element-wise multiplication and $\sigma$ is the sigmoid activation function
+ Film modulation: A Feature-wise Linear Modulation proposed [here](https://arxiv.org/pdf/1709.07871.pdf)




In [1]:
from analysis_utils import *

/home/prtos/anaconda3/envs/invivo/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
res_folder = os.path.abspath('./dataset_repr/results/')

## Analysis of the validation lost and the meta-test performances

In [3]:
resdata, test_filenames = get_result_summaries(res_folder)
display(resdata.head())
print(resdata.shape)
widgets = get_widgets(resdata, filter_names=['condition_on']) # the metric widget is implicit
drawing_func = lambda **x: get_heatmaps(data=resdata,
                       heat_column_names=['num_layers'],
                       heat_row_names=['arch'], **x)

ipywidgets.interact(drawing_func, **widgets)

,events_file,condition_on,conditioner_mode,arch,num_layers,mse,pcc,r2,tboard_folder,val_loss
0,/home/prtos/workspace/code/phd/few_shot_regres...,train,fusion,relation_net_mean,2,0.564425,0.649444,0.422210,/home/prtos/workspace/code/phd/few_shot_regres...,0.540217
1,/home/prtos/workspace/code/phd/few_shot_regres...,both,fusion,set2set,3,0.039063,0.980749,0.958226,/home/prtos/workspace/code/phd/few_shot_regres...,0.038360
2,/home/prtos/workspace/code/phd/few_shot_regres...,train,fusion,simple_attention,1,0.559035,0.654817,0.428815,/home/prtos/workspace/code/phd/few_shot_regres...,0.534206
3,/home/prtos/workspace/code/phd/few_shot_regres...,both,fusion,multihead_attention,3,0.198537,0.883683,0.787229,/home/prtos/workspace/code/phd/few_shot_regres...,0.206233
4,/home/prtos/workspace/code/phd/few_shot_regres...,train,fusion,set2set,2,0.628107,0.594471,0.352865,/home/prtos/workspace/code/phd/few_shot_regres...,0.616748


(35, 10)


interactive(children=(Dropdown(description='Metric', options=('mse', 'r2', 'pcc', 'val_loss'), value='mse'), D…

<function __main__.<lambda>(**x)>

In [4]:
widgets = get_test_fitting_curves_widgets(resdata, test_filenames) # the metric widget is implicit
drawing_func = lambda **x: plot_fitting_curves(resdata, **x)

ipywidgets.interact(drawing_func, **widgets)

interactive(children=(Dropdown(description='Arch', options=('relation_net_mean', 'None', 'multihead_attention'…

<function __main__.<lambda>(**x)>

## Closing thougths and next steps

### Results
+ Overall conditionning on the task is very helpfull for this meta-dataset
+ No clear winner between alternatives to obtain the task representations
+ The method of conditionning seems to be very important though: 
    + The fusion net seems to outperform the two other alternatives explored
    + The poor performance of Film  may be related to a gradient propagation problem

### Next steps and advices from A. Lacoste
- Film multiple layers 
- More complex network and test squeeze and excite again
- Eval extrapolation: not mandatory
- Encoding of pairs of train samples and do self attention. Transformer and relationNet (Priority)
- query and support set vocab instead of train/test vocab
- Batch norm for adam regrularization
- lr cv for on set on hp
- Baselines: Deep Prior, Conditional neural process(CNP), Bayesian MAML (Priority)
- Send test fitting curves to Alex and include them in your notbooks at the next steps